# Load data

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [2]:
# Load and split data
filename = '../data/bankruptcy.csv'
df = pd.read_csv(filename, sep=',', index_col=False)

# Drop column (always same value)
df.drop(columns=[' Net Income Flag'], inplace=True)

# Drop two outlier rows (encoding errors)
df.drop(df[df[' Revenue per person'] > 1].index, inplace=True)

# Split into X, Y
values = df.values
X, Y = values[:, 1:], values[:, 0]
feature_names = list(df.columns)[1:]

# Train/val/test split
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=0)
X_train, X_val, Y_train, Y_val = train_test_split(
    X_train, Y_train, test_size=0.2, random_state=0)

In [3]:
# Neural network pre-processing
enc = OneHotEncoder(sparse=False)
Y_train_oh = enc.fit_transform(np.expand_dims(Y_train, -1))
Y_val_oh = enc.fit_transform(np.expand_dims(Y_val, -1))
Y_test_oh = enc.transform(np.expand_dims(Y_test, -1))

ss = StandardScaler()
ss.fit(X_train)
X_train_std = ss.transform(X_train)
X_val_std = ss.transform(X_val)
X_test_std = ss.transform(X_test)

# Set up imputer

In [4]:
import torch
import fastshap_torch
from fastshap_torch import Surrogate

In [5]:
device = torch.device('cuda', 3)
surrogate = torch.load('../models/bankruptcy_surrogate.pt').eval().to(device)
num_features = X_train.shape[1]
surr = Surrogate(surrogate, num_features)

# FastSHAP

In [6]:
import torch
import torch.nn as nn
from fastshap_torch import FastSHAP

# Test samples number

- I'm trying with no penalty because the ablation experiment showed that no penalty was better, at least for 32 samples with paired sampling. But these results are not so good...

In [7]:
for n_samples in (1, 4, 16, 32, 48, 64, 96):
    # Set up explainer model
    explainer = nn.Sequential(
        nn.Linear(num_features, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 2 * num_features)).to(device)

    # Set up FastSHAP wrapper
    fastshap = FastSHAP(explainer, surr, normalization='additive', link=nn.Softmax(dim=-1))

    # Train
    fastshap.train(
        X_train_std,
        X_val_std[:500],
        batch_size=32,
        lr=2e-4,
        min_lr=1e-4,
        num_samples=n_samples,
        max_epochs=200,
        eff_lambda=0,
        paired_sampling=False,
        validation_samples=128,
        validation_seed=123,
        lookback=10,
        verbose=False)

    # Print performance
    print('Best val loss = {:.8f}'.format(min(fastshap.loss_list)))

    # Save model
    modifier = 'samples={}'.format(n_samples)
    explainer.cpu()
    torch.save(explainer, '../models/bankruptcy_explainer {} nopenalty.pt'.format(modifier))

Best val loss = 0.00804849
Best val loss = 0.00135096
Best val loss = 0.00017779
Best val loss = 0.00016724
Best val loss = 0.00016727
Best val loss = 0.00017060
Best val loss = 0.00017634


In [14]:
for n_samples in (4, 16, 32, 48, 64, 96):
# for n_samples in (32, 64):
    # Set up explainer model
    explainer = nn.Sequential(
        nn.Linear(num_features, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 2 * num_features)).to(device)

    # Set up FastSHAP wrapper
    fastshap = FastSHAP(explainer, surr, normalization='additive', link=nn.Softmax(dim=-1))

    # Train
    fastshap.train(
        X_train_std,
        X_val_std[:500],
        batch_size=32,
        lr=1e-3,
        min_lr=1e-4,
        num_samples=n_samples,
        max_epochs=200,
        eff_lambda=0,
        paired_sampling=True,
        validation_samples=128,
        validation_seed=123,
        lookback=10,
        verbose=False)

    # Print performance
    print('Best val loss = {:.8f}'.format(min(fastshap.loss_list)))

    # Save model
    modifier = 'paired_samples={}'.format(n_samples)
    explainer.cpu()
    torch.save(explainer, '../models/bankruptcy_explainer {} nopenalty.pt'.format(modifier))

Best val loss = 0.00141389
Best val loss = 0.00017716
Best val loss = 0.00016697
Best val loss = 0.00016942
Best val loss = 0.00017018
Best val loss = 0.00016867


# Test other parameters

In [15]:
# Set up explainer model
explainer = nn.Sequential(
    nn.Linear(num_features, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 2 * num_features)).to(device)

# Set up FastSHAP wrapper
fastshap = FastSHAP(explainer, surr, normalization='additive', link=nn.Softmax(dim=-1))

# Train
fastshap.train(
    X_train_std,
    X_val_std[:500],
    batch_size=32,
    lr=1e-3,
    min_lr=1e-4,
    num_samples=32,
    max_epochs=200,
    eff_lambda=0,
    paired_sampling=True,
    validation_samples=128,
    validation_seed=123,
    lookback=10,
    verbose=False)

# Print performance
print('Best val loss = {:.8f}'.format(min(fastshap.loss_list)))

# Save model
modifier = 'nopenalty'
explainer.cpu()
torch.save(explainer, '../models/bankruptcy_explainer {}.pt'.format(modifier))

Best val loss = 0.00016934


In [16]:
# Set up explainer model
explainer = nn.Sequential(
    nn.Linear(num_features, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 2 * num_features)).to(device)

# Set up FastSHAP wrapper
fastshap = FastSHAP(explainer, surr, normalization=None, link=nn.Softmax(dim=-1))

# Train
fastshap.train(
    X_train_std,
    X_val_std[:500],
    batch_size=32,
    lr=1e-3,
    min_lr=1e-4,
    num_samples=32,
    max_epochs=200,
    eff_lambda=0.1,
    paired_sampling=True,
    validation_samples=128,
    validation_seed=123,
    lookback=10,
    verbose=False)

# Print performance
print('Best val loss = {:.8f}'.format(min(fastshap.loss_list)))

# Save model
modifier = 'nonormalization'
explainer.cpu()
torch.save(explainer, '../models/bankruptcy_explainer {}.pt'.format(modifier))

Best val loss = 0.00027661


In [17]:
# Set up explainer model
explainer = nn.Sequential(
    nn.Linear(num_features, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 2 * num_features)).to(device)

# Set up FastSHAP wrapper
fastshap = FastSHAP(explainer, surr, normalization=None, link=nn.Softmax(dim=-1))

# Train
fastshap.train(
    X_train_std,
    X_val_std[:500],
    batch_size=32,
    lr=1e-3,
    min_lr=1e-4,
    num_samples=32,
    max_epochs=200,
    eff_lambda=0,
    paired_sampling=True,
    validation_samples=128,
    validation_seed=123,
    lookback=10,
    verbose=False)

# Print performance
print('Best val loss = {:.8f}'.format(min(fastshap.loss_list)))

# Save model
modifier = 'nopenalty nonormalization'
explainer.cpu()
torch.save(explainer, '../models/bankruptcy_explainer {}.pt'.format(modifier))

Best val loss = 0.00019273
